In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Embedding, Dense
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


In [3]:
# Preprocessing: lowercase, stopwords, lemmatization
def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)

In [4]:
from tensorflow.keras.models import load_model

# If saved as an HDF5 file
model = load_model('bilstm_model.h5')

In [7]:
# Load data
data = pd.read_csv('train.csv')  
# remove emptyrecords
data = data.dropna()

data['Review'] = data['Review'].apply(preprocess_text)

X = data['Review']
y = data['overall'].values - 1

# tokenization and padding
embedding_dim = 100
max_len = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_seq, maxlen=max_len, padding='post')
vocab_size = len(tokenizer.word_index) + 1
epochs = 10
batch_size = 64

KeyboardInterrupt: 

In [ ]:
test_data = pd.read_csv('test.csv') 
test_data = test_data.dropna()

test_data['Review'] = test_data['Review'].apply(preprocess_text)

test_sequences = tokenizer.texts_to_sequences(test_data['Review'])
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post')

test_predictions = model.predict(test_padded)
predicted_labels = np.argmax(test_predictions, axis=1)

submission_df = pd.DataFrame({'Review': test_data['Review'], 'Predicted': predicted_labels + 1}) 

3864/3864 [==============================] - 61s 16ms/step


ImportError: cannot import name 'SequenceNotStr' from 'pandas._typing' (/Users/fayazbadubhai/anaconda3/lib/python3.11/site-packages/pandas/_typing.py)

In [ ]:
submission_df.to_csv('smote_lstm.csv', index=False)